# Easy experiment setup

This notebook demonstrates some convenience functions for coming up with the
experiments needed to do GST on a new gateset.

In [1]:
import pygsti
import pygsti.algorithms.germselection as germsel
import pygsti.algorithms.fiducialselection as fidsel
import pygsti.construction as constr

Let's use the 1-qubit $X(\pi/2)$, $Y(\pi/2)$, $I$ gateset already packaged with pyGSTi, but pretend that
we don't have precomputed germs and fiducials available.

In [2]:
import pygsti.construction.std1Q_XYI as std
gs_target = std.gs_target

## Hands-off

We begin by demonstrating the most hands-off approach.

We can generate a germ set simply by providing the target gateset.

In [3]:
germs = germsel.generate_germs(gs_target)

We can also generate preparation and measurement fiducials, although in addition to the
target gateset we also need to identify gates from the target gateset not to use in the
fiducials (in this case we don't want the identity gate included in our fiducials).

In [4]:
gatesToOmit = ['Gi']
prepFiducials, measFiducials = fidsel.generate_fiducials(gs_target, gatesToOmit)

Complete initial fiducial set succeeds.
Now searching for best fiducial set.
Final fiducial set succeeds.
Complete initial fiducial set succeeds.
Now searching for best fiducial set.
Final fiducial set succeeds.


/Users/jongros/src/python/pyGSTi/packages/pygsti/algorithms/scoring.py:38: RuntimeWarning: divide by zero encountered in true_divide
  score = sum(1. / _np.abs(input_array))


Now that we have germs and fiducials, we can construct the list of experiments we need to perform in
order to do GST. The only new things to provide at this point are the sizes for the experiments we want
to perform (in this case we want to perform between 0 and 256 gates between fiducial pairs, going up
by a factor of 2 at each stage).

In [5]:
maxLengths = [0] + [2**n for n in range(8 + 1)]
listOfExperiments = constr.make_lsgst_experiment_list(gs_target.gates.keys(), prepFiducials,
                                                      measFiducials, germs, maxLengths)

The list of `GateString` that the previous function gave us isn't necessarily the most readable
form to present the information in, so we can write the experiment list out to an empty data
file to be filled in after the experiments are performed.

In [6]:
pygsti.io.write_empty_dataset("tutorial_files/EasyDataTemplate.txt", listOfExperiments)

That's it! You can now take the data this file is asking for and come back to analyze the data.

## More control

There are many ways you can assume more control over the experiment design process. We'll only demonstrate
a few here, but all options are discussed in the documentation for the various functions we've used.

There are a number of different algorithms available for germ selection. You can choose a non-default
algorithm by specifying the `algorithm` keyword argument.

Each of the available algorithms has a set of keyword arguments of its own with which you
can more precisely specify how you want it to behave. These keyword arguments can be passed
as a dictionary to `generate_germs` through the keyword argument `algorithm_kwargs`.

In [7]:
slackGerms = germsel.generate_germs(gs_target, algorithm='slack', algorithm_kwargs={'maxIter': 15})

In [9]:
len(slackGerms)

181

We got an unwieldy germ set, but that's because we set the maximum number of iterations very low
for run-time considerations.

Fiducial selection can be controlled in much the same way.

In [8]:
graspPrepFids, graspMeasFids = fidsel.generate_fiducials(gs_target, gatesToOmit, algorithm='grasp',
                                                         algorithm_kwargs={'alpha': 0.2})

/Users/jongros/src/python/pyGSTi/packages/pygsti/algorithms/scoring.py:38: RuntimeWarning: divide by zero encountered in true_divide
  score = sum(1. / _np.abs(input_array))
